In [3]:
import pickle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [4]:
# print(torch.is_gpu_)

In [5]:

# Read the pickle object
with open("../../dataset/pickles/LOC3.pickle", "rb") as f:
    data = pickle.load(f)

# Convert the data to a pandas dataframe
df = pd.DataFrame(data)

# Encode the class labels
label_encoder = LabelEncoder()
df['class_label'] = label_encoder.fit_transform(df['class_label'])

# Define the maximum sequence length
max_len = 0
for i in df['lengths']:
    max_len=max(max_len,len(i))

# Pad the sequences to the maximum length
sequences = np.array(df['lengths'])
padded_sequences = np.zeros((len(sequences), max_len))
for i, sequence in enumerate(sequences):
    padded_sequences[i, :len(sequence)] = sequence

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(padded_sequences, df['class_label'], test_size=0.2)

# Convert the data to PyTorch tensors
X_train = torch.from_numpy(X_train).to(torch.float32)
y_train = torch.from_numpy(y_train.values).long()
X_val = torch.from_numpy(X_val).to(torch.float32)
y_val = torch.from_numpy(y_val.values).long()
# check if split is stratified
print("Training set class distribution:")
print(y_train.unique(return_counts=True)) 
print("Validation set class distribution:")
print(y_val.unique(return_counts=True))
print(y_train.unique().shape)
print(y_val.unique().shape)

Training set class distribution:
(tensor([   0,    1,    2,  ..., 1497, 1498, 1499]), tensor([82, 84, 87,  ..., 86, 73, 81]))
Validation set class distribution:
(tensor([   0,    1,    2,  ..., 1497, 1498, 1499]), tensor([25, 21, 19,  ..., 19, 29, 24]))
torch.Size([1500])
torch.Size([1500])


In [7]:
# Define a custom dataset
class SequenceDataset(Dataset):
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels
    
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        sequence = self.sequences[index]
        label = self.labels[index]
        return sequence, label


In [8]:

# Initialize the model
input_dim=0
for i in X_train:
    input_dim = max(input_dim,len(i))
# input_dim = len(X_train[0])
hidden_dim = 128
output_dim = 1500
learning_rate = 1e-3
print(input_dim)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
# Define the dataloaders
train_dataset = SequenceDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataset = SequenceDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


1213


In [9]:

# Define the model architecture
class RNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, batch_size):
        super(RNN, self).__init__()
        self.batch_size=batch_size
        # simple 1d conv 
        self.conv1 = nn.Conv1d(1, 8, 3, padding=1)
        self.bn1 = nn.BatchNorm1d(8)
        self.relu = nn.ReLU()        
        
        # parameters for the RNN
        self.hidden_dim = hidden_dim
        self.input_dim = input_dim
        self.output_dim = output_dim

        # define the RNN
        # self.rnn = nn.LSTM(8*input_dim, hidden_dim, batch_first=True)
        self.rnn = nn.RNN(8*input_dim, hidden_dim, batch_first=True)
        # define the output layer
        self.fc = nn.Linear(hidden_dim, output_dim)    
        self.h = self.init_hidden(self.batch_size)    
        self.flatten = nn.Flatten()
        # self.fc = nn.Linear(8*input_dim, output_dim)
        
    def init_hidden(self, bsz):
        # initialize the hidden state
        self.h=torch.zeros(1, bsz, self.hidden_dim).to(device)
    
    def forward(self, x):
        # reshape the input
        # x = x.view(self.batch_size, self.input_dim, -1)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.flatten(x)
        # initialize the hidden state
        self.init_hidden(x.shape[0])
        # get RNN unit outputs
        out, h = self.rnn(x.unsqueeze(1), self.h)
        # get the output for the last time step
        out = self.fc(out[:, -1, :])
        return out

In [10]:
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
rnn = RNN(input_dim, hidden_dim, output_dim, batch_size=64)
rnn = rnn.to(device)
criterion = criterion.to(device)

optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)


Using device: cuda:6


In [11]:
from sklearn.metrics import precision_recall_fscore_support
def get_precision_recall_f1(y_true, y_pred):
    """Calculate precision, recall, F1 score for each class.
    Args:
        y_true (list): list of true labels
        y_pred (list): list of predicted labels
    Returns:
        dict: dictionary with precision, recall, F1 score for each class
    """
    # Calculate precision, recall, F1 score for each class
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

num_epochs = 100
# optimizer = optimizer.to(device)
for epoch in range(num_epochs):
    running_loss = 0.0
    running_corrects = 0
    rnn.train()
    for sequences, labels in train_loader:
        optimizer.zero_grad()
        labels = labels.to(device)
        sequences = sequences.to(device)
        sequences = sequences.unsqueeze(1)
        outputs = rnn(sequences)
        
        # print(sequences.shape)
        # print(outputs.shape)
        # print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * sequences.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)
    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = running_corrects.double() /len(train_dataset)
    print('Train Loss: {:.4f} Train Acc: {:.4f} %'.format(epoch_loss, epoch_acc*100))
    
    # test the model
    rnn.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        total_preds = []
        total_labels = []
        for sequences, labels in val_loader:
            sequences = sequences.to(device)
            labels = labels.to(device)
            sequences = sequences.unsqueeze(1)
            outputs = rnn(sequences)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            total_preds.append(predicted)
            total_labels.append(labels)
        
        print('Test Accuracy of the model on the test sequences: {} %'.format(100 * correct / total))
        total_preds = torch.cat(total_preds, dim=0)
        total_labels = torch.cat(total_labels, dim=0)
        print(get_precision_recall_f1(total_labels.cpu(), total_preds.cpu()))

Train Loss: 4.1951 Train Acc: 20.1071 %
Test Accuracy of the model on the test sequences: 37.53832252234808 %
{'precision': 0.3829803069670158, 'recall': 0.37638851138244306, 'f1': 0.33455669707443275}


/home/anirudha/miniconda3/envs/765/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Loss: 2.3572 Train Acc: 47.0124 %
Test Accuracy of the model on the test sequences: 52.78665246716365 %
{'precision': 0.5372059739651895, 'recall': 0.5285931198022131, 'f1': 0.5033229602582767}


/home/anirudha/miniconda3/envs/765/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Loss: 1.7410 Train Acc: 59.3234 %
Test Accuracy of the model on the test sequences: 60.81582599154484 %
{'precision': 0.620229293136035, 'recall': 0.6076250153107966, 'f1': 0.5886198272872815}


...

/home/anirudha/miniconda3/envs/765/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Loss: 0.2566 Train Acc: 91.3535 %
Test Accuracy of the model on the test sequences: 81.28570045502953 %
{'precision': 0.8207537058149361, 'recall': 0.8145911327468274, 'f1': 0.8075583907885462}


/home/anirudha/miniconda3/envs/765/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Loss: 0.2582 Train Acc: 91.1808 %
Test Accuracy of the model on the test sequences: 81.48578436118372 %
{'precision': 0.8234383057847551, 'recall': 0.8159869226482562, 'f1': 0.8094023912229559}


/home/anirudha/miniconda3/envs/765/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print(total_preds.unique().shape)
print(total_labels.unique().shape)
print("Accuracy:{:.4f}".format(torch.sum(total_preds == total_labels).item()/len(total_labels)))
print(total_preds.shape)
print("Precision:{:.4f} Recall:{:.4f} F1:{:.4f}".format(get_precision_recall_f1(total_labels.cpu(), total_preds.cpu())['precision'], get_precision_recall_f1(total_labels.cpu(), total_preds.cpu())['recall'], get_precision_recall_f1(total_labels.cpu(), total_preds.cpu())['f1']))

torch.Size([1467])
torch.Size([1500])
Accuracy:0.7917
torch.Size([30987])
Precision:0.8021 Recall:0.7924 F1:0.7865


/home/anirudha/miniconda3/envs/765/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anirudha/miniconda3/envs/765/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/anirudha/miniconda3/envs/765/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(r

In [ ]:
# Evaluate the model
running_loss = 0.0
running_corrects = 0
rnn.eval()
with torch.no_grad():
    for sequences, labels in val_loader:
        labels = labels.to(device)
        sequences = sequences.to(device)
        sequences = sequences.unsqueeze(1)
        outputs = rnn(sequences)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * sequences.size(0)
        _, preds = torch.max(outputs, 1)
        running_corrects += torch.sum(preds == labels)
epoch_loss = running_loss / len(val_dataset)
epoch_acc = running_corrects.double() / len(val_dataset)
print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(epoch_loss, epoch_acc))


Val Loss: 1.7257 Val Acc: 0.6621


In [ ]:
accuracy = model.predict(test_data)
print("Accuracy: ", accuracy,"%")

Accuracy: 90%


In [ ]:
a = torch.tensor([[1]])